In [ ]:
!pip install f5-tts gradio ruaccent runorm transformers torch torchaudio huggingface_hub soundfile pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 50.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# НАЖАТЬ RESTART SESSION

In [ ]:
#!/usr/bin/env python3
import time
import re
import os
import gc
import tempfile
import traceback
from pydub import AudioSegment
from pathlib import Path

import gradio as gr
import numpy as np
import soundfile as sf
import torch
import torchaudio
from huggingface_hub import hf_hub_download, snapshot_download
from ruaccent import RUAccent
from runorm import RUNorm
from f5_tts.infer.utils_infer import (
    infer_process,
    load_model,
    load_vocoder,
    preprocess_ref_audio_text,
    remove_silence_for_generated_wav,
    save_spectrogram,
    tempfile_kwargs,
)
from f5_tts.model import DiT

# 🔹 Добавьте эти импорты для Whisper
from transformers import WhisperProcessor, WhisperForConditionalGeneration

MODELS_CONFIG = {
    "ESpeech-TTS-1_SFT-95K": {
        "repo": "ESpeech/ESpeech-TTS-1_SFT-95K",
        "model_file": "espeech_tts_95k.pt",
        "vocab_file": "vocab.txt",
        "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    },
    "ESpeech-TTS-1_RL-V1": {
        "repo": "ESpeech/ESpeech-TTS-1_RL-V1",
        "model_file": "espeech_tts_rlv1.pt",
        "vocab_file": "vocab.txt",
        "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    },
    "ESpeech-TTS-1_RL-V2": {
        "repo": "ESpeech/ESpeech-TTS-1_RL-V2",
        "model_file": "espeech_tts_rlv2.pt",
        "vocab_file": "vocab.txt",
        "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    },
    "F5TTS_v1_Base_v2_Misha24-10": {
        "repo": "Misha24-10/F5-TTS_RUSSIAN",
        "model_file": "F5TTS_v1_Base_v2/model_last_inference.safetensors",
        "vocab_file": "F5TTS_v1_Base/vocab.txt",
        "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    },
    "F5TTS_v1_Base_1.25M_SWivid": {
    "repo": "SWivid/F5-TTS",
    "model_file": "F5TTS_v1_Base/model_1250000.safetensors",
    "vocab_file": "F5TTS_v1_Base/vocab.txt",
    "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    },
}

# Глобальные переменные для моделей
loaded_models = {}
loaded_vocoder = None
current_model_name = None
stop_generation = False
remember_seed = False
last_seed = -1

# 🔴 НОВОЕ: Словарь для локальных моделей
local_models_config = {}

def check_cuda_availability():
    """Check if CUDA is available and raise error if not"""
    if not torch.cuda.is_available():
        raise RuntimeError("CUDA is not available! This application requires GPU with CUDA support.")
    print(f"CUDA is available. Using device: {torch.cuda.get_device_name()}")

def add_local_model(model_path, vocab_path, model_name):
    """Добавить локальную модель в конфигурацию"""
    global local_models_config, MODELS_CONFIG
    
    if not model_path or not vocab_path or not model_name:
        return gr.update(value="❌ Заполните все поля", visible=True)
    
    # Проверяем существование файлов
    if not os.path.exists(model_path):
        return gr.update(value="❌ Файл модели не найден", visible=True)
    
    if not os.path.exists(vocab_path):
        return gr.update(value="❌ Файл словаря не найден", visible=True)
    
    # Добавляем модель в локальную конфигурацию
    local_models_config[model_name] = {
        "repo": "local",
        "model_file": model_path,
        "vocab_file": vocab_path,
        "model_cfg": dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
    }
    
    # Обновляем общий конфиг моделей
    updated_models_config = {**MODELS_CONFIG, **local_models_config}
    
    return gr.update(
        choices=list(updated_models_config.keys()),
        value=model_name
    ), gr.update(value=f"✅ Локальная модель '{model_name}' добавлена", visible=True)

def load_model_with_progress(model_name, progress_callback=None):
    """Загрузка модели с индикатором прогресса"""
    global loaded_models, current_model_name, stop_generation, MODELS_CONFIG, local_models_config
    
    # Проверка флага остановки
    if stop_generation:
        return None
    
    # Объединяем конфиги моделей
    all_models_config = {**MODELS_CONFIG, **local_models_config}
    
    # Валидация входных параметров
    if not model_name or model_name not in all_models_config:
        raise ValueError(f"Unknown model: {model_name}")
    
    # Если модель уже загружена, возвращаем её
    if model_name in loaded_models:
        return loaded_models[model_name]
    
    # Получаем конфигурацию модели
    config = all_models_config[model_name]
    
    # 🔴 ДЛЯ ЛОКАЛЬНЫХ МОДЕЛЕЙ - ПРОВЕРЯЕМ ФАЙЛЫ
    if config['repo'] == "local":
        model_path = config['model_file']
        vocab_path = config['vocab_file']
        
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Local model file not found: {model_path}")
        if not os.path.exists(vocab_path):
            raise FileNotFoundError(f"Local vocab file not found: {vocab_path}")
            
        print(f"Loading local model from: {model_path}")
        
    else:
        # Старая логика для моделей из HuggingFace
        if progress_callback:
            progress_callback("🔍 Поиск модели...", 0.1)
        
        # Проверяем доступность CUDA
        check_cuda_availability()

        model_path = None
        vocab_path = None
        
        # Обновляем прогресс
        if progress_callback:
            progress_callback("📥 Загрузка модели...", 0.3)
        
        # Скачиваем файлы модели
        try:
            model_path = hf_hub_download(repo_id=config['repo'], filename=config['model_file'])
            vocab_path = hf_hub_download(repo_id=config['repo'], filename=config['vocab_file'])
            print(f"Downloaded model to {model_path}")
            print(f"Downloaded vocab to {vocab_path}")
        except Exception as e:
            print("hf_hub_download failed:", e)
            
        # Проверяем флаг остановки
        if stop_generation:
            return None
            
        # Если не удалось скачать отдельные файлы, пробуем скачать всю модель
        if model_path is None or vocab_path is None:
            try:
                local_dir = f"cache_{config['repo'].replace('/', '_')}"
                if progress_callback:
                    progress_callback("📦 Скачивание полной модели...", 0.5)
                snapshot_dir = snapshot_download(repo_id=config['repo'], cache_dir=None, local_dir=local_dir)
                possible_model = os.path.join(snapshot_dir, config['model_file'])
                possible_vocab = os.path.join(snapshot_dir, config['vocab_file'])
                if os.path.exists(possible_model):
                    model_path = possible_model
                if os.path.exists(possible_vocab):
                    vocab_path = possible_vocab
            except Exception as e:
                print("snapshot_download failed:", e)
                
        # Проверяем флаг остановки
        if stop_generation:
            return None
            
        # Проверяем, что файлы существуют
        if not model_path or not os.path.exists(model_path):
            raise FileNotFoundError(f"Model file not found: {model_path}")
        if not vocab_path or not os.path.exists(vocab_path):
            raise FileNotFoundError(f"Vocab file not found: {vocab_path}")

    # 🔴 ОБЩАЯ ЛОГИКА ЗАГРУЗКИ МОДЕЛИ
    if progress_callback:
        progress_callback("🔄 Загрузка модели в память...", 0.7)
    
    # Загружаем модель
    print(f"Loading model from: {model_path}")
    model = load_model(DiT, config['model_cfg'], model_path, vocab_file=vocab_path)

    # Проверяем флаг остановки
    if stop_generation:
        return None
        
    # Обновляем прогресс
    if progress_callback:
        progress_callback("🚀 Перенос модели на GPU...", 0.9)
    
    # Перемещаем модель на GPU
    device = torch.device("cuda")
    model.to(device)
    print(f"Model loaded and moved to CUDA: {device}")

    # Сохраняем модель в кэш
    loaded_models[model_name] = model
    current_model_name = model_name

    # Финальное обновление прогресса
    if progress_callback:
        progress_callback("✅ Модель загружена!", 1.0)
    
    return model

def ensure_model(model_name, progress_callback=None):
    """Ensure model is loaded with progress tracking"""
    global loaded_models, MODELS_CONFIG, local_models_config
    
    if not model_name:
        raise ValueError("Model name must be specified")
    
    # Объединяем конфиги моделей
    all_models_config = {**MODELS_CONFIG, **local_models_config}
    
    if model_name not in all_models_config:
        raise ValueError(f"Unknown model: {model_name}")
    
    # Если модель уже загружена, возвращаем её
    if model_name in loaded_models:
        return loaded_models[model_name]
    
    # Иначе загружаем с отслеживанием прогресса
    return load_model_with_progress(model_name, progress_callback)

def ensure_vocoder():
    global loaded_vocoder
    if loaded_vocoder is not None:
        return loaded_vocoder

    # Проверка CUDA
    check_cuda_availability()

    print("⏳ Loading vocoder...")
    
    try:
        loaded_vocoder = load_vocoder()
        device = torch.device("cuda")
        loaded_vocoder.to(device)
        print("✅ Vocoder loaded successfully")
        return loaded_vocoder
    except Exception as e:
        print(f"❌ Vocoder loading failed: {e}")
        raise e

def stop_generation_process():
    """Остановка генерации"""
    global stop_generation
    stop_generation = True
    torch.cuda.empty_cache()
    gc.collect()
    return "🛑 Генерация остановлена"

def reset_stop_flag():
    """Сброс флага остановки"""
    global stop_generation
    stop_generation = False

# --- Загрузка RUNorm ---
print("Loading RUNorm (text normalizer)...")
try:
    normalizer = RUNorm()
    normalizer.load(
        model_size="medium",
        device="cuda" if torch.cuda.is_available() else "cpu",
        workdir="./local_cache"  # Кэширование моделей
    )
    print("RUNorm loaded.")
except Exception as e:
    print(f"Failed to load RUNorm: {e}")
    exit(1)

# --- Check CUDA ---
print("Loading RUAccent...")
accentizer = RUAccent()
accentizer.load(omograph_model_size='turbo3.1', use_dictionary=True, tiny_mode=False)
print("RUAccent loaded.")


print("Loading Whisper ASR model (supports Russian and English)...")
try:
    from transformers import WhisperProcessor, WhisperForConditionalGeneration

    ASR_MODEL_NAME = "openai/whisper-medium"  # Быстро и точно. Можно: "openai/whisper-base", "medium", "large-v3"
    asr_processor = WhisperProcessor.from_pretrained(ASR_MODEL_NAME)
    asr_model = WhisperForConditionalGeneration.from_pretrained(ASR_MODEL_NAME)
    asr_model.to("cuda" if torch.cuda.is_available() else "cpu")
    print("Whisper model loaded. Supports Russian, English, and many other languages.")
except Exception as e:
    print(f"Failed to load Whisper model: {e}")
    asr_processor = None
    asr_model = None

def transcribe_audio(audio_path):
    if not audio_path or asr_processor is None or asr_model is None:
        return ""

    try:
        # 1. Загрузка аудио
        waveform, sample_rate = torchaudio.load(audio_path)

        # 2. Ресемплинг в 16 кГц
        if sample_rate != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
            waveform = resampler(waveform)

        # 3. Моно
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        # 4. В numpy
        input_audio = waveform.squeeze().numpy()

        # 5. Обработка: получаем input_features (без паддинга)
        inputs = asr_processor(
            input_audio,
            sampling_rate=16000,
            return_tensors="pt",
            # ❗ Убираем padding и attention_mask — будем падить вручную
        )

        input_features = inputs.input_features  # [1, 80, T], где T < 3000

        # 🔹 🔥 РУЧНОЙ ПАДДИНГ ДО 3000
        current_length = input_features.shape[-1]
        if current_length < 3000:
            pad_length = 3000 - current_length
            input_features = torch.nn.functional.pad(input_features, (0, pad_length), mode='constant', value=0)
        elif current_length > 3000:
            input_features = input_features[..., :3000]  # обрезаем, если вдруг больше

        print("Input features shape:", input_features.shape)  # Должно быть [1, 80, 3000]

        # 6. Переносим на GPU
        input_features = input_features.to(asr_model.device)

        # 7. Генерация
        with torch.no_grad():
            generated_ids = asr_model.generate(
                input_features=input_features,
                language=None,
                task="transcribe",
            )

        # 8. Децодинг
        transcription = asr_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        return transcription.strip().capitalize() + "."

    except Exception as e:
        print("ASR transcription failed:", e)
        return "Ошибка распознавания."
# --- Конец функции ---

def validate_and_transcribe_audio(audio_path):
    if not audio_path:
        return None, "", gr.update(value="", visible=False)

    try:
        # Просто транскрибируем аудио без проверок
        transcription = transcribe_audio(audio_path)
        
        # ✅ Всегда возвращаем аудиофайл
        return audio_path, transcription, gr.update(value="", visible=False)
        
    except Exception as e:
        print("Validation failed:", e)
        return None, "", gr.update(value="⚠️ **Ошибка при проверке аудио**", visible=True)

# Check CUDA availability at startup
print("Checking CUDA availability...")
try:
    check_cuda_availability()
except RuntimeError as e:
    print(f"FATAL ERROR: {e}")
    exit(1)

# --- Обработка текста: СНАЧАЛА runorm, ПОТОМ ruaccent ---
def process_text_with_accent(text):
    if not text or not text.strip():
        return text

    # --- 1. Проверка: есть ли ударения (+) в словах? ---
    # Ищем + между буквами: "прив+ет", "выс+окий", но не "5 + 3"
    has_accent_in_words = bool(re.search(r'\w\+\w', text))
    # ---

    # --- 2. Если НЕТ ударений → нормализуем через runorm ---
    if not has_accent_in_words:
        try:
            # Заменяем + на временный символ, чтобы runorm не трогал ударения
            text_safe = text.replace("+", "⚡")
            text_normalized = normalizer.norm(text_safe)
            # Возвращаем + обратно (на всякий случай)
            text = text_normalized.replace("⚡", "+")
        except Exception as e:
            print(f"runorm.norm() failed: {e}")
            text = text
    else:
        # Ударения уже есть → пропускаем runorm
        text = text
    # ---

    # --- 3. Расстановка ударений через ruaccent (если нет +) ---
    if '+' in text:
        return text
    try:
        return accentizer.process_all(text)
    except Exception as e:
        print(f"ruaccent failed: {e}")
        return text

def process_texts_only(ref_text, gen_text):
    # Блокировка обработки путей к файлам
    if ref_text and isinstance(ref_text, str):
        if any(bad in ref_text for bad in ["/tmp/", ".wav", ".mp3", ".ogg", ".flac", "/home/", "/root/"]):
            ref_text = ""  # или оставьте пустым
    return (
        process_text_with_accent(ref_text),
        process_text_with_accent(gen_text)
    )

def get_current_seed_display():
    """Получить текущее значение seed для отображения"""
    global last_seed
    # Если last_seed еще не установлен, генерируем случайный
    if last_seed == -1:
        last_seed = np.random.randint(0, 2**31 - 1)
        print(f"🎲 Инициализирован начальный seed: {last_seed}")
    return last_seed

# 🔴 ИСПРАВЛЕННАЯ ФУНКЦИЯ: Обновление статуса загрузки модели
def update_model_loading_status(model_name):
    """Обновление статуса загрузки модели"""
    global loaded_models, MODELS_CONFIG, local_models_config
    
    if not model_name:
        return gr.update(value="<div class='model-status model-error'>❌ Модель не выбрана</div>", visible=True)
    
    # Объединяем конфиги моделей
    all_models_config = {**MODELS_CONFIG, **local_models_config}
    
    if model_name not in all_models_config:
        return gr.update(value=f"<div class='model-status model-error'>❌ Модель {model_name} не найдена</div>", visible=True)
    
    # 🔴 ИСПРАВЛЕНИЕ: Проверяем, загружена ли модель в данный момент
    if model_name in loaded_models:
        model_type = "🔧 Локальная" if all_models_config[model_name]['repo'] == "local" else "🌐 HuggingFace"
        return gr.update(value=f"<div class='model-status model-loaded'>✅ {model_name} ({model_type}) загружена</div>", visible=True)
    else:
        # 🔴 ИСПРАВЛЕНИЕ: Показываем статус "не загружена" только если модель не загружена
        model_type = "🔧 Локальная" if all_models_config[model_name]['repo'] == "local" else "🌐 HuggingFace"
        return gr.update(value=f"<div class='model-status model-loading'>🔄 {model_name} ({model_type}) не загружена</div>", visible=True)

# 🔴 ИСПРАВЛЕННАЯ ФУНКЦИЯ: Загрузка модели с обновлением статуса
def load_model_with_status(model_name, progress=gr.Progress()):
    """Загрузка модели с обновлением статуса"""
    global loaded_models
    
    if not model_name:
        return gr.update(value="<div class='model-status model-error'>❌ Модель не выбрана</div>", visible=True)
    
    # Если модель уже загружена, просто возвращаем статус
    if model_name in loaded_models:
        return update_model_loading_status(model_name)
    
    try:
        # Показываем статус загрузки
        progress(0.1, desc="🔄 Загрузка модели...")
        
        # Загружаем модель
        model = load_model_with_progress(model_name, lambda msg, p: progress(p, desc=msg))
        
        if model:
            # Успешная загрузка
            return update_model_loading_status(model_name)
        else:
            return gr.update(value="<div class='model-status model-error'>❌ Загрузка отменена</div>", visible=True)
            
    except Exception as e:
        error_msg = f"❌ Ошибка загрузки: {str(e)}"
        print(f"Model loading error: {traceback.format_exc()}")
        return gr.update(value=f"<div class='model-status model-error'>{error_msg}</div>", visible=True)

def on_model_select(model_name):
    """Обработчик выбора модели в dropdown"""
    global MODELS_CONFIG, local_models_config
    
    if not model_name:
        return gr.update(value="<div class='model-status model-error'>❌ Модель не выбрана</div>", visible=True)
    
    # Объединяем конфиги моделей
    all_models_config = {**MODELS_CONFIG, **local_models_config}
    
    if model_name not in all_models_config:
        return gr.update(value=f"<div class='model-status model-error'>❌ Модель {model_name} не найдена</div>", visible=True)
    
    # 🔴 ИСПРАВЛЕНИЕ: Просто показываем текущий статус без автоматической загрузки
    return update_model_loading_status(model_name)

def synthesize(
    ref_audio,
    ref_text,
    gen_text,
    remove_silence,
    seed_input_value,
    remember_seed_checkbox,
    model_choice,
    cross_fade_duration=0.15,
    nfe_step=32,
    speed=1.0,
    sway_sampling_coef=-1,
    cfg_strength=2,
    audio_format="wav",
    bitrate="192k",
    progress=gr.Progress()
):
    global stop_generation, last_seed, remember_seed
    
    # 🔴 ОБНОВЛЯЕМ ФЛАГ ЗАПОМИНАНИЯ СИДА
    remember_seed = remember_seed_checkbox
    
    reset_stop_flag()
    
    # 🔴 СРАЗУ ОБНОВЛЯЕМ СТАТУС МОДЕЛИ
    current_model_status = update_model_loading_status(model_choice)
    
    # 🔴 РАННЯЯ ПРОВЕРКА STOP_FLAG
    if stop_generation:
        gr.Warning("🛑 Генерация остановлена")
        return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status
    
    # 🔄 ОБНОВЛЯЕМ СТАТУС ЗАГРУЗКИ
    progress(0.05, desc="🔄 Подготовка...")
    
    # 🎯 ПРОВЕРКА ВХОДНЫХ ДАННЫХ
    if not ref_audio:
        gr.Warning("❌ Пожалуйста, предоставьте референсное аудио")
        return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status
    
    if not gen_text or not gen_text.strip():
        gr.Warning("❌ Пожалуйста, введите текст для генерации")
        return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status
        
    if not ref_text or not ref_text.strip():
        gr.Warning("❌ Пожалуйста, предоставьте референсный текст")
        return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status

    # 🔐 ЗАЩИТА ОТ ПУТЕЙ К ФАЙЛАМ
    if gen_text and isinstance(gen_text, str):
        if any(x in gen_text for x in ["/tmp/", ".wav", ".mp3", ".ogg", ".flac"]):
            gr.Warning("⚠️ Поле 'Text to Generate' содержит путь к файлу. Пожалуйста, введите текст вручную.")
            return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status

    # 🔴 ПРОВЕРКА STOP_FLAG
    if stop_generation:
        gr.Warning("🛑 Генерация остановлена")
        return None, None, ref_text, gen_text, gr.update(value=get_current_seed_display()), current_model_status

    # 🎲 УСТАНОВКА SEED - КЛЮЧЕВОЕ ИЗМЕНЕНИЕ!
    current_seed = seed_input_value
    if seed_input_value is None or seed_input_value < 0 or seed_input_value > 2**31 - 1:
        # 🔴 ГЕНЕРИРУЕМ СЛУЧАЙНЫЙ СИД
        current_seed = np.random.randint(0, 2**31 - 1)
        print(f"🎲 Сгенерирован случайный seed: {current_seed}")
    
    # 🔴 ЗАПОМИНАЕМ СИД ЕСЛИ ВКЛЮЧЕНА ГАЛОЧКА
    if remember_seed:
        last_seed = current_seed
        print(f"💾 Seed сохранен: {current_seed}")
    
    torch.manual_seed(int(current_seed))
    
    # 🔴 ВЫВОДИМ ИНФОРМАЦИЮ О СИДЕ
    print(f"🎲 Используется seed: {current_seed} (запомнить: {remember_seed})")

    # 📝 ОБРАБОТКА ТЕКСТА
    progress(0.08, desc="📝 Обработка текста...")

    # --- ✅ Используем тексты как есть ---
    processed_ref_text = ref_text
    processed_gen_text = gen_text

    # --- ✅ Инициализация processed_ref_text_final ДО try ---
    processed_ref_text_final = ref_text  

    try:
        # --- 1. ОТЛОЖЕННАЯ ЗАГРУЗКА МОДЕЛЕЙ ---
        progress(0.1, desc="🔍 Проверка моделей...")
        
        # 🔴 ПРОВЕРКА STOP_FLAG
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
        
        # 📥 ЗАГРУЗКА ОСНОВНОЙ МОДЕЛИ
        progress(0.15, desc="📥 Загрузка модели TTS...")
        model = ensure_model(model_choice, lambda msg, p: progress(p, desc=msg))
        
        # 🔴 ОБНОВЛЯЕМ СТАТУС ПОСЛЕ ЗАГРУЗКИ МОДЕЛИ
        if model:
            current_model_status = update_model_loading_status(model_choice)
        
        # 🔴 ПРОВЕРКА STOP_FLAG
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
        
        # 🔊 ЗАГРУЗКА ВОКОДЕРА (ОТЛОЖЕННАЯ)
        progress(0.2, desc="🔊 Загрузка вокодера...")
        vocoder = ensure_vocoder()
        
        # 🔴 ПРОВЕРКА STOP_FLAG
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
            
    except Exception as e:
        error_msg = f"❌ Ошибка загрузки моделей: {str(e)}"
        gr.Warning(error_msg)
        print(f"Model loading error: {traceback.format_exc()}")
        return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

    # 🎯 ПРОВЕРКА УСПЕШНОЙ ЗАГРУЗКИ
    if model is None or vocoder is None:
        gr.Warning("❌ Не удалось загрузить модели")
        return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

    device = torch.device("cuda")

    try:
        # --- 2. ПРЕДОБРАБОТКА АУДИО И ТЕКСТА ---
        progress(0.3, desc="🔧 Предобработка аудио...")
        
        # 🔴 ПРОВЕРКА STOP_FLAG
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
        
        try:
            ref_audio_proc, processed_ref_text_final = preprocess_ref_audio_text(
                ref_audio,
                processed_ref_text,
                show_info=gr.Info
            )
        except Exception as e:
            error_msg = f"❌ Ошибка предобработки: {str(e)}"
            gr.Warning(error_msg)
            traceback.print_exc()
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
        
        # --- 3. ГЕНЕРАЦИЯ АУДИО ---
        progress(0.5, desc="🎵 Генерация аудио...")
        
        # 🔴 ПРОВЕРКА STOP_FLAG
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
        
        try:
            final_wave, final_sample_rate, combined_spectrogram = infer_process(
                ref_audio_proc,
                processed_ref_text_final,
                processed_gen_text,
                model,
                vocoder,
                cross_fade_duration=cross_fade_duration,
                nfe_step=nfe_step,
                speed=speed,
                sway_sampling_coef=sway_sampling_coef,
                cfg_strength=cfg_strength,
                show_info=gr.Info,
                progress=progress,
            )
        except Exception as e:
            error_msg = f"❌ Ошибка генерации аудио: {str(e)}"
            gr.Warning(error_msg)
            traceback.print_exc()
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
        
        # 🔴 ПРОВЕРКА STOP_FLAG
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

        # --- 4. УДАЛЕНИЕ ТИШИНЫ (ОПЦИОНАЛЬНО) ---
        if remove_silence:
            progress(0.7, desc="🔇 Удаление тишины...")
            
            # 🔴 ПРОВЕРКА STOP_FLAG
            if stop_generation:
                gr.Warning("🛑 Генерация остановлена")
                return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status
            
            try:
                with tempfile.NamedTemporaryFile(suffix=".wav", **tempfile_kwargs) as f:
                    temp_path = f.name
                    sf.write(temp_path, final_wave, final_sample_rate)
                    remove_silence_for_generated_wav(temp_path)
                    final_wave_tensor, _ = torchaudio.load(temp_path)
                    final_wave = final_wave_tensor.squeeze().cpu().numpy()
            except Exception as e:
                print("⚠️ Удаление тишины не удалось:", e)
                # Не прерываем выполнение, продолжаем с оригинальным аудио

        # --- 5. ЭКСПОРТ В ФОРМАТ ---
        progress(0.8, desc="💾 Сохранение результата...")
        
        # 🔴 ПРОВЕРКА STOP_FLAG
        if stop_generation:
            gr.Warning("🛑 Генерация остановлена")
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

        # 🎵 ПОДГОТОВКА АУДИО ДЛЯ ЭКСПОРТА
        safe_kwargs = {k: v for k, v in tempfile_kwargs.items() if k != 'delete'}

        with tempfile.NamedTemporaryFile(suffix=f".{audio_format}", delete=False, **safe_kwargs) as tmp:
            temp_output_path = tmp.name

        wave = final_wave
        if wave.ndim == 1:
            channels = 1
        else:
            channels = min(wave.shape[0], 2)
            wave = wave[:channels].T

        # 🔧 НОРМАЛИЗАЦИЯ АУДИО
        if np.max(np.abs(wave)) == 0:
            wave_int16 = np.zeros(wave.shape, dtype=np.int16)
        else:
            wave_int16 = np.int16(wave / np.max(np.abs(wave)) * 32767)

        # 💾 СОХРАНЕНИЕ В ВЫБРАННОМ ФОРМАТЕ
        audio_segment = AudioSegment(
            wave_int16.tobytes(),
            frame_rate=final_sample_rate,
            sample_width=2,
            channels=channels
        )

        try:
            if audio_format == "mp3":
                audio_segment.export(temp_output_path, format="mp3", bitrate=bitrate)
            elif audio_format == "ogg":
                audio_segment.export(temp_output_path, format="ogg", bitrate=bitrate)
            elif audio_format == "flac":
                audio_segment.export(temp_output_path, format="flac")
            else:
                audio_segment.export(temp_output_path, format="wav")
        except Exception as e:
            error_msg = f"❌ Ошибка экспорта аудио: {str(e)}"
            gr.Warning(error_msg)
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

        # 📁 СОЗДАНИЕ ФИНАЛЬНОГО ФАЙЛА
        timestamp = int(time.time())
        safe_text = f"audio_{timestamp}"
        final_output_path = os.path.join(os.path.dirname(temp_output_path), f"{safe_text}.{audio_format}")

        try:
            if os.path.exists(final_output_path):
                os.remove(final_output_path)
            os.rename(temp_output_path, final_output_path)
        except Exception as e:
            error_msg = f"❌ Ошибка создания файла: {str(e)}"
            gr.Warning(error_msg)
            return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

        # --- 6. СОХРАНЕНИЕ СПЕКТРОГРАММЫ ---
        progress(0.9, desc="📊 Сохранение спектрограммы...")
        
        spectrogram_path = None
        try:
            with tempfile.NamedTemporaryFile(suffix=".png", **tempfile_kwargs) as tmp_spectrogram:
                spectrogram_path = tmp_spectrogram.name
                save_spectrogram(combined_spectrogram, spectrogram_path)
        except Exception as e:
            print("⚠️ Сохранение спектрограммы не удалось:", e)
            # Не прерываем выполнение, продолжаем без спектрограммы

        progress(1.0, desc="✅ Готово!")
        
        # 🎉 ВОЗВРАТ УСПЕШНОГО РЕЗУЛЬТАТА
        return (
            final_output_path,
            spectrogram_path,
            processed_ref_text_final,
            processed_gen_text,
            gr.update(value=current_seed),  # 🔴 ВОЗВРАЩАЕМ РЕАЛЬНЫЙ СИД
            current_model_status  # 🔴 ВОЗВРАЩАЕМ АКТУАЛЬНЫЙ СТАТУС МОДЕЛИ
        )

    except Exception as e:
        error_msg = f"❌ Неожиданная ошибка: {str(e)}"
        print(f"Unexpected error in synthesize: {traceback.format_exc()}")
        gr.Warning(error_msg)
        return None, None, processed_ref_text, processed_gen_text, gr.update(value=current_seed), current_model_status

    finally:
        # 🧹 ОЧИСТКА ПАМЯТИ (ЕСЛИ НЕ БЫЛО ОСТАНОВКИ)
        if not stop_generation:
            try:
                torch.cuda.empty_cache()
                gc.collect()
            except Exception as e:
                print("⚠️ Ошибка при очистке памяти:", e)

with gr.Blocks(title="ESpeech-TTS", css="""
    .error-markdown {
        color: red !important;
        font-weight: bold;
        text-align: center;
        font-size: 18px !important;
        margin: 10px 0;
        animation: fadeIn 0.3s, fadeOut 0.3s 4.7s forwards;
    }
    .model-status {
        padding: 10px;
        border-radius: 5px;
        margin: 10px 0;
        text-align: center;
        font-weight: bold;
    }
    .model-loaded {
        background: #d4edda;
        color: #155724;
        border: 1px solid #c3e6cb;
    }
    .model-error {
        background: #f8d7da;
        color: #721c24;
        border: 1px solid #f5c6cb;
    }
    .model-loading {
        background: #fff3cd;
        color: #856404;
        border: 1px solid #ffeaa7;
    }
    .loading-status {
        padding: 10px;
        margin: 10px 0;
        border-radius: 5px;
        text-align: center;
    }
    .local-model-input {
        background: #f8f9fa;
        padding: 15px;
        border-radius: 8px;
        border: 1px solid #dee2e6;
        margin: 10px 0;
    }
    @keyframes fadeIn { from { opacity: 0; } to { opacity: 1; } }
    @keyframes fadeOut { from { opacity: 1; } to { opacity: 0; } }
    """) as app:
    gr.Markdown("# ESpeech-TTS")
    gr.Markdown("💡 **Совет:** Добавьте символ '+' для ударения (например, 'прив+ет')")
    gr.Markdown("🎲 **Совет:** Seed отвечает за вариативность генерации. Вы можете подбирать для себя удачное значение или каждый раз использовать случайный seed")
    gr.Markdown("🚀 **CUDA Required:** This application requires GPU with CUDA support")

    # 🔴 НОВЫЙ БЛОК: Добавление локальных моделей
    with gr.Accordion("🔧 Добавить локальную модель", open=False):
        with gr.Row():
            local_model_path = gr.Textbox(
                label="Путь к файлу модели",
                placeholder="hf://SWivid/F5-TTS/F5TTS_v1_Base/model_1250000.safetensors или /path/to/model.safetensors",
                scale=3
            )
            local_vocab_path = gr.Textbox(
                label="Путь к файлу словаря", 
                placeholder="hf://SWivid/F5-TTS/F5TTS_v1_Base/vocab.txt или /path/to/vocab.txt",
                scale=3
            )
            local_model_name = gr.Textbox(
                label="Имя модели",
                placeholder="My Local Model",
                scale=2
            )
        with gr.Row():
            add_local_model_btn = gr.Button("➕ Добавить локальную модель", variant="secondary")
            local_model_status = gr.HTML(visible=False)

    # --- 🔴 ИСПРАВЛЕННЫЙ СТАТУС МОДЕЛИ ---
    initial_model = "ESpeech-TTS-1_SFT-95K"
    initial_status = update_model_loading_status(initial_model)

    model_status = gr.HTML(
        value=initial_status["value"],
        visible=initial_status["visible"]
    )

    with gr.Row():
        with gr.Column():
            # --- Выбор модели ---
            model_choice = gr.Dropdown(
                choices=list(MODELS_CONFIG.keys()),
                value=initial_model,
                label="Выберите модель",
                info="🌐 HuggingFace модели или 🔧 локальные модели"
            )
            
            # --- 🔴 ИСПРАВЛЕННАЯ КНОПКА ЗАГРУЗКИ МОДЕЛИ ---
            load_model_btn = gr.Button("🔄 Загрузить выбранную модель", variant="secondary")
            
        with gr.Column():
            ref_audio_input = gr.Audio(label="Reference Audio", type="filepath")
            ref_text_input = gr.Textbox(
                label="Reference Text",
                lines=2,
                placeholder="Text corresponding to reference audio"
            )
            # Новое: текст ошибки
            ref_audio_warning = gr.Markdown("", visible=False, elem_classes="error-markdown")
        with gr.Column():
            gen_text_input = gr.Textbox(
                label="Text to Generate",
                lines=5,
                max_lines=20,
                placeholder="Enter text to synthesize..."
            )

    # --- 🔴 ИСПРАВЛЕННЫЕ ОБРАБОТЧИКИ ДЛЯ ЗАГРУЗКИ МОДЕЛИ ---
    def on_model_select(model_name):
        """Обработчик выбора модели в dropdown"""
        if model_name:
            return update_model_loading_status(model_name)
        return gr.update(value="<div class='model-status model-error'>❌ Модель не выбрана</div>", visible=True)

    # 🔴 НОВЫЙ ОБРАБОТЧИК: Добавление локальной модели
    add_local_model_btn.click(
        fn=add_local_model,
        inputs=[local_model_path, local_vocab_path, local_model_name],
        outputs=[model_choice, local_model_status]
    )

    # Автозагрузка модели при выборе - ТОЛЬКО СТАТУС
    model_choice.change(
        fn=on_model_select,
        inputs=model_choice,
        outputs=model_status
    )

    # Кнопка принудительной загрузки
    load_model_btn.click(
        fn=load_model_with_status,
        inputs=[model_choice],
        outputs=model_status
    )

    # --- Подключение автоматического распознавания ✅---
    ref_audio_input.change(
        fn=validate_and_transcribe_audio,
        inputs=ref_audio_input,
        outputs=[ref_audio_input, ref_text_input, ref_audio_warning]
    )

    process_text_btn = gr.Button("✏️ Нормализация текста", variant="secondary")

    with gr.Accordion("Advanced Settings", open=False):
        with gr.Row():
            # 🔴 ИСПРАВЛЕНИЕ: показываем реальный seed
            seed_input = gr.Number(label="Seed (random)", value=get_current_seed_display(), precision=0)
            remember_seed_checkbox = gr.Checkbox(label="💾 Запомнить этот seed", value=False)
            remove_silence = gr.Checkbox(label="Remove Silences", value=False)
        with gr.Row():
            speed_slider = gr.Slider(label="Speed", minimum=0.3, maximum=2.0, value=1.0, step=0.1)
            nfe_slider = gr.Slider(label="NFE Steps", minimum=4, maximum=64, value=48, step=2)
        with gr.Row():
            cross_fade_slider = gr.Slider(label="Cross-Fade Duration (s)", minimum=0.0, maximum=1.0, value=0.15, step=0.01)
            sway_sampling_slider = gr.Slider(label="Sway Sampling Coef", minimum=-1, maximum=1, value=-1, step=0.1)
        with gr.Row():
            cfg_strength_slider = gr.Slider(label="CFG Strength", minimum=0.5, maximum=5.0, value=2.0, step=0.1)
        with gr.Row():
            audio_format = gr.Radio(["wav", "mp3", "ogg", "flac"], label="Формат", value="wav")
            bitrate = gr.Radio(["128k", "192k", "320k"], label="Битрейт (mp3/ogg)", value="192k", visible=lambda fmt: fmt in ["mp3", "ogg"])

    # --- Обработчик изменения формата ---
    def update_bitrate_visibility(audio_format):
        return gr.update(visible=audio_format in ["mp3", "ogg"])

    audio_format.change(
        update_bitrate_visibility,
        inputs=audio_format,
        outputs=bitrate
    )

    # --- 🔴 ДОБАВЛЕНО: Кнопка остановки ---
    with gr.Row():
        generate_btn = gr.Button("🎤 Generate Speech", variant="primary", size="lg")
        stop_btn = gr.Button("🛑 Stop Generation", variant="stop", size="lg")

    with gr.Row():
        audio_output = gr.Audio(label="🎧 Аудио", type="filepath")
        spectrogram_output = gr.Image(label="Spectrogram", type="filepath")

    process_text_btn.click(
        process_texts_only,
        inputs=[ref_text_input, gen_text_input],
        outputs=[ref_text_input, gen_text_input]
    )

    # --- 🔴 ДОБАВЛЕНО: Обработчик остановки ---
    stop_btn.click(
        fn=stop_generation_process,
        outputs=None,
        queue=False
    )

    # 🔴 ИСПРАВЛЕННЫЙ ВЫЗОВ СИНТЕЗА С ОБНОВЛЕНИЕМ СТАТУСА
    generate_btn.click(
        synthesize,
        inputs=[
            ref_audio_input,
            ref_text_input,
            gen_text_input,
            remove_silence,
            seed_input,
            remember_seed_checkbox,
            model_choice,
            cross_fade_slider,
            nfe_slider,
            speed_slider,
            sway_sampling_slider,
            cfg_strength_slider,
            audio_format,
            bitrate,
        ],
        outputs=[
            audio_output, 
            spectrogram_output, 
            ref_text_input, 
            gen_text_input, 
            seed_input,
            model_status  # 🔴 ДОБАВЛЯЕМ ОБНОВЛЕНИЕ СТАТУСА МОДЕЛИ
        ]
    )

if __name__ == "__main__":
    app.launch(share=True)

Checking CUDA availability...
CUDA is available. Using device: Tesla T4
Preloading model...
CUDA is available. Using device: Tesla T4
Trying to download model file 'espeech_tts_rlv2.pt' and 'vocab.txt' from hub 'ESpeech/ESpeech-TTS-1_RL-V2'


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Downloaded model to /root/.cache/huggingface/hub/models--ESpeech--ESpeech-TTS-1_RL-V2/snapshots/f582b6e5897fe8a5835059405a8439d13bdf7684/espeech_tts_rlv2.pt
Downloaded vocab to /root/.cache/huggingface/hub/models--ESpeech--ESpeech-TTS-1_RL-V2/snapshots/f582b6e5897fe8a5835059405a8439d13bdf7684/vocab.txt
Loading model from: /root/.cache/huggingface/hub/models--ESpeech--ESpeech-TTS-1_RL-V2/snapshots/f582b6e5897fe8a5835059405a8439d13bdf7684/espeech_tts_rlv2.pt

vocab :  /root/.cache/huggingface/hub/models--ESpeech--ESpeech-TTS-1_RL-V2/snapshots/f582b6e5897fe8a5835059405a8439d13bdf7684/vocab.txt
token :  custom
model :  /root/.cache/huggingface/hub/models--ESpeech--ESpeech-TTS-1_RL-V2/snapshots/f582b6e5897fe8a5835059405a8439d13bdf7684/espeech_tts_rlv2.pt 

Model loaded and moved to CUDA: cuda
Model preloaded.
Loading RUAccent...
RUAccent loaded.
Preloading vocoder...
CUDA is available. Using device: Tesla T4
Loading vocoder...
Download Vocos from huggingface charactr/vocos-mel-24khz
Vocoder